In [ ]:
import torch
from PIL import Image
from torchvision.transforms import v2
import torchvision.transforms as T
import glob
import os
import posixpath


In [ ]:
# flip all images horizontally and save to output folder
cwd = os.getcwd()

source_filters = ['./datasample/s/cc/*.jpg', './datasample/m/cc/*.jpg', './datasample/b/cc/*.jpg']
destination_folders = ['./datasample/s/cc/flip', './datasample/m/cc/flip', './datasample/b/cc/flip']

for i in range(len(source_filters)):
    source_filter = source_filters[i]
    source_label_filter = source_filter.replace(".jpg", "_label.jpg")
    destination_folder = destination_folders[i]
    print("\n", len(glob.glob(source_filter)), source_filter, source_label_filter, destination_folder)
    if not os.path.isdir(os.path.join(cwd,destination_folder)):
        os.mkdir(os.path.join(cwd,destination_folder))
    
    all_images = sorted(glob.glob(source_filter),
                        key = lambda j: (len(posixpath.basename(j)), j))
    all_labels = sorted(glob.glob(source_label_filter),
                        key = lambda j: (len(posixpath.basename(j)), j))
    all_inputs = []
    
    for j in all_images:
        if j in all_labels:
            continue 
        else:
            all_inputs.append(j)
    
    for input, label in zip(all_inputs, all_labels):
        print('.', end='')
        image = Image.open(input)
        mask = Image.open(label)
        transform = T.Compose([T.PILToTensor()])
    
        tensor_im = transform(image)
        tensor_ma = transform(mask)
        
        transforms = v2.Compose([
            v2.RandomHorizontalFlip(p=1.0),
        ])
        out_im = transforms(tensor_im)
        out_ma = transforms(tensor_ma)
        
        base1, ext1 = os.path.splitext(os.path.basename(input))  
        ext1 = ext1.lower()  
        new_path1 = os.path.join(destination_folder, f"{base1}f{ext1}")
        
        base2, ext2 = os.path.splitext(os.path.basename(label)) 
        ext2 = ext2.lower()  
        new_path2 = os.path.join(destination_folder, f"{base2[:-6]}f_label{ext2}")
        
        trans = T.ToPILImage()
        ima = trans(out_im)
        mas = trans(out_ma)
        ima.save(new_path1)
        mas.save(new_path2)
    
print(".")
print("finished")